In [ ]:
#importing necessary libraries
import numpy as np
import tensorflow as tf
from keras import models 
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
#Run this cell only if your data resides on Google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Set base_path to the location where the data and results of your project
#reside
base_path = '/content/gdrive/MyDrive/HPT/'

#Loading Features and Creating Models

##Functions to extract features for CE-MRI and HMS datasets

In [ ]:
def load_ce_mri_features(strategy):
  #load the already extracted features
  Meningioma = np.load(base_path + 'Features/' + strategy + '/CE-MRI/Meningioma_Features.npy')
  Glioma = np.load(base_path + 'Features/' + strategy + '/CE-MRI/Glioma_Features.npy')
  Pituitary = np.load(base_path + 'Features/' + strategy + '/CE-MRI/Pituitary_Features.npy')
  
  #create labels (0. Meningioma, 1. Glioma, 2. Pituitary)
  Meningioma_label = np.zeros((Meningioma.shape[0], 1), dtype=np.uint8)
  
  Glioma_label = np.zeros((Glioma.shape[0], 1), dtype=np.uint8)
  Glioma_label[:, 0] = 1

  Pituitary_label = np.zeros((Pituitary.shape[0], 1), dtype=np.uint8)
  Pituitary_label[:, 0] = 2

  #concatenating all the classes (features and their labels)
  TrainX = np.concatenate((Meningioma, Glioma, Pituitary), axis=0)
  TrainY = np.concatenate((Meningioma_label, Glioma_label, Pituitary_label), axis=0)

  TrainY_one_hot = to_categorical(TrainY, dtype ="uint8")

  #split into train and test sets (80:20)
  TrainX_Split, TestX_split, TrainY_Split, TestY_Split = train_test_split(TrainX, TrainY_one_hot, test_size=0.2, random_state=42, stratify=TrainY_one_hot)

  return TrainX_Split, TestX_split, TrainY_Split, TestY_Split;

In [ ]:
def load_hms_features(strategy):
  Cereb = np.load(base_path + 'Features/' + strategy + '/HMS/Cerebrovascular_Features.npy')
  Deg = np.load(base_path + 'Features/' + strategy + '/HMS/Degenerative_Features.npy')
  Inf = np.load(base_path + 'Features/' + strategy + '/HMS/Inflammatory_Features.npy')
  Neo = np.load(base_path + 'Features/' + strategy + '/HMS/Neoplastic_Features.npy')
  normal = np.load(base_path + 'Features/' + strategy + '/HMS/Normal_Features.npy')

  #creating labels (0. Cerebrovascular, 1. Degenerative, 2. Inflammatory,
  #3. Neoplastic, 4. Normal)
  Cereb_label = np.zeros((689, 1), dtype=np.uint8)
  Deg_label = np.zeros((370, 1), dtype=np.uint8)
  Deg_label[:, 0] = 1
  Inf_label = np.zeros((463, 1), dtype=np.uint8)
  Inf_label[:, 0] = 2
  Neo_label = np.zeros((868, 1), dtype=np.uint8)
  Neo_label[:, 0] = 3
  Normal_label = np.zeros((314, 1), dtype=np.uint8)
  Normal_label[:, 0] = 4
  
  #concatenating all the classes (features and their labels)
  TrainX = np.concatenate((Cereb,Deg, Inf, Neo, normal), axis=0)
  TrainY = np.concatenate((Cereb_label, Deg_label,Inf_label, Neo_label, Normal_label), axis=0)
  
  TrainY_one_hot = to_categorical(TrainY, dtype ="uint8")

  #split into train and test sets (80:20)
  TrainX_Split, TestX_split, TrainY_Split, TestY_Split = train_test_split(TrainX, TrainY_one_hot, test_size=0.2, random_state=42, stratify=TrainY_one_hot)
  
  return TrainX_Split, TestX_split, TrainY_Split, TestY_Split;

##Functions to create models for all the strategies and datasets

In [ ]:
def create_model_1_full_architecture(dataset):
  if dataset=='HMS':
    model_1 = Sequential([
      Dense(64, activation='relu', input_shape=(2048,)),
      Dense(32, activation='relu'),
      Dense(5, activation='softmax')
      ])
  elif dataset == 'CE-MRI':
    model_1 = Sequential([
      Dense(64, activation='relu', input_shape=(2048,)),
      Dense(32, activation='relu'),
      Dense(3, activation='softmax')
      ])
  
  return model_1

In [ ]:
def create_model_2_full_architecture(dataset):
  if dataset=="HMS":
    model_2 = Sequential([
      Dense(512, activation='relu', input_shape=(2048,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(128, activation='relu'),
      Dense(5, activation='softmax')
    ])
  elif dataset=="CE-MRI":
    model_2 = Sequential([
      Dense(512, activation='relu', input_shape=(2048,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(128, activation='relu'),
      Dense(3, activation='softmax')
    ])
  
  return model_2

In [ ]:
def create_model_1_F1B(dataset):
  if dataset=='HMS':
    model_1 = Sequential([
      Dense(64, activation='relu', input_shape=(256,)),
      Dense(32, activation='relu'),
      Dense(5, activation='softmax')
      ])
  elif dataset == 'CE-MRI':
    model_1 = Sequential([
      Dense(64, activation='relu', input_shape=(256,)),
      Dense(32, activation='relu'),
      Dense(3, activation='softmax')
      ])
  
  return model_1

In [ ]:
def create_model_2_F1B(dataset):
  if dataset=="HMS":
    model_2 = Sequential([
      Dense(512, activation='relu', input_shape=(256,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(128, activation='relu'),
      Dense(5, activation='softmax')
    ])
  elif dataset=="CE-MRI":
    model_2 = Sequential([
      Dense(512, activation='relu', input_shape=(256,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(128, activation='relu'),
      Dense(3, activation='softmax')
    ])

  return model_2

In [ ]:
def create_model_1_F2B(dataset):
  if dataset=='HMS':
    model_1 = Sequential([
      Dense(64, activation='relu', input_shape=(512,)),
      Dense(32, activation='relu'),
      Dense(5, activation='softmax')
      ])
  elif dataset == 'CE-MRI':
    model_1 = Sequential([
      Dense(64, activation='relu', input_shape=(512,)),
      Dense(32, activation='relu'),
      Dense(3, activation='softmax')
      ])
  
  return model_1

In [ ]:
def create_model_2_F2B(dataset):
  if dataset=="HMS":
    model_2 = Sequential([
      Dense(512, activation='relu', input_shape=(512,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(128, activation='relu'),
      Dense(5, activation='softmax')
    ])
  elif dataset=="CE-MRI":
    model_2 = Sequential([
      Dense(512, activation='relu', input_shape=(512,)),
      Dense(256, activation='relu'),
      Dense(256, activation='relu'),
      Dense(128, activation='relu'),
      Dense(3, activation='softmax')
    ])

  return model_2

#Conduct Trainings, Save Checkpoints and Evaluate on Test Sets

##Functions to train and evaluate models



In [ ]:
def train_model(strategy, dataset, model_number):
  if dataset == "CE-MRI":
    TrainX, TestX, TrainY, TestY = load_ce_mri_features(strategy)
  elif dataset == "HMS":
    TrainX, TestX, TrainY, TestY = load_hms_features(strategy)

  #create model
  if strategy == "DA_TF_F1B" or strategy == "ImageNet_TF_F1B":
    if model_number == 1:
      model = create_model_1_F1B(dataset)
    else:
      model = create_model_2_F1B(dataset)
  elif strategy == "DA_TF_F2B" or strategy == "ImageNet_TF_F2B":
    if model_number == 1:
      model = create_model_1_F2B(dataset)
    else:
      model = create_model_2_F2B(dataset)
  else:
    if model_number == 1:
      model = create_model_1_full_architecture(dataset)
    else:
      model = create_model_2_full_architecture(dataset)

  #compile the model
  model.compile(optimizer = optimizers.Adam(learning_rate=0.00005),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
  
  #create callback
  callbacks = []
  checkpoint_path = base_path + 'Results/' + strategy + '/' + dataset + '/Model ' + str(model_number) + '/'
  callbacks.append(ModelCheckpoint(checkpoint_path + 'checkpoint-{epoch}.h5'))
  
  #train the model
  history = model.fit(TrainX, TrainY, batch_size=64, epochs=500, callbacks=callbacks)
#strategy == "DA_TF_F1B" or strategy == "ImageNet_TF_F1B":

In [ ]:
def evaluate_model(strategy, dataset, model_number):
  test_set_auc = np.zeros((500,))
  checkpoint_path = base_path + 'Results/' + strategy + '/' + dataset + '/Model ' + str(model_number) + '/'
  
  for i in range(1, 501):
    loaded_model = models.load_model(checkpoint_path + 'checkpoint-' + str(i) + '.h5')
    y_pred = loaded_model.predict(TestX, steps=TestX.shape[0], verbose=0)
    test_set_auc[i] = roc_auc_score(TestY, y_pred, average="macro")

  #save test set AUC for all the epochs (1 to 500)
  np.save(checkpoint_path + 'TestAUC.npy', test_set_auc)

##Train and evaluate models for each strategy and dataset by calling functions

DA_TF_F1B

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("DA_TF_F1B", "CE-MRI", 1)
evaluate_model("DA_TF_F1B", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("DA_TF_F1B", "CE-MRI", 2)
evaluate_model("DA_TF_F1B", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("DA_TF_F1B", "HMS", 1)
evaluate_model("DA_TF_F1B", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("DA_TF_F1B", "HMS", 2)
evaluate_model("DA_TF_F1B", "HMS", 2)

ImageNet_TF_F1B

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("ImageNet_TF_F1B", "CE-MRI", 1)
evaluate_model("ImageNet_TF_F1B", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("ImageNet_TF_F1B", "CE-MRI", 2)
evaluate_model("ImageNet_TF_F1B", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("ImageNet_TF_F1B", "HMS", 1)
evaluate_model("ImageNet_TF_F1B", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("ImageNet_TF_F1B", "HMS", 2)
evaluate_model("ImageNet_TF_F1B", "HMS", 2)

DA_TF_F2B

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("DA_TF_F2B", "CE-MRI", 1)
evaluate_model("DA_TF_F2B", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("DA_TF_F2B", "CE-MRI", 2)
evaluate_model("DA_TF_F2B", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("DA_TF_F2B", "HMS", 1)
evaluate_model("DA_TF_F2B", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("DA_TF_F2B", "HMS", 2)
evaluate_model("DA_TF_F2B", "HMS", 2)

ImageNet_TF_F2B

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("ImageNet_TF_F2B", "CE-MRI", 1)
evaluate_model("ImageNet_TF_F2B", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("ImageNet_TF_F2B", "CE-MRI", 2)
evaluate_model("ImageNet_TF_F2B", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("ImageNet_TF_F2B", "HMS", 1)
evaluate_model("ImageNet_TF_F2B", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("ImageNet_TF_F2B", "HMS", 2)
evaluate_model("ImageNet_TF_F2B", "HMS", 2)

DA

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("DA", "CE-MRI", 1)
evaluate_model("DA", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("DA", "CE-MRI", 2)
evaluate_model("DA", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("DA", "HMS", 1)
evaluate_model("DA", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("DA", "HMS", 2)
evaluate_model("DA", "HMS", 2)

DA_L1SB

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("DA_L1SB", "CE-MRI", 1)
evaluate_model("DA_L1SB", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("DA_L1SB", "CE-MRI", 2)
evaluate_model("DA_L1SB", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("DA_L1SB", "HMS", 1)
evaluate_model("DA_L1SB", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("DA_L1SB", "HMS", 2)
evaluate_model("DA_L1SB", "HMS", 2)

DA_L2SB

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("DA_L2SB", "CE-MRI", 1)
evaluate_model("DA_L2SB", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("DA_L2SB", "CE-MRI", 2)
evaluate_model("DA_L2SB", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("DA_L2SB", "HMS", 1)
evaluate_model("DA_L2SB", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("DA_L2SB", "HMS", 2)
evaluate_model("DA_L2SB", "HMS", 2)

DA_L1SB_PFT

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("DA_L1SB_PFT", "CE-MRI", 1)
evaluate_model("DA_L1SB_PFT", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("DA_L1SB_PFT", "CE-MRI", 2)
evaluate_model("DA_L1SB_PFT", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("DA_L1SB_PFT", "HMS", 1)
evaluate_model("DA_L1SB_PFT", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("DA_L1SB_PFT", "HMS", 2)
evaluate_model("DA_L1SB_PFT", "HMS", 2)

DA_L2SB_PFT

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("DA_L2SB_PFT", "CE-MRI", 1)
evaluate_model("DA_L2SB_PFT", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("DA_L2SB_PFT", "CE-MRI", 2)
evaluate_model("DA_L2SB_PFT", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("DA_L2SB_PFT", "HMS", 1)
evaluate_model("DA_L2SB_PFT", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("DA_L2SB_PFT", "HMS", 2)
evaluate_model("DA_L2SB_PFT", "HMS", 2)

ImageNet

In [ ]:
#######################CE-MRI (Model 1)#######################
train_model("ImageNet", "CE-MRI", 1)
evaluate_model("ImageNet", "CE-MRI", 1)

In [ ]:
#######################CE-MRI (Model 2)#######################
train_model("ImageNet", "CE-MRI", 2)
evaluate_model("ImageNet", "CE-MRI", 2)

In [ ]:
#######################HMS (Model 1)#######################
train_model("ImageNet", "HMS", 1)
evaluate_model("ImageNet", "HMS", 1)

In [ ]:
#######################HMS (Model 2)#######################
train_model("ImageNet", "HMS", 2)
evaluate_model("ImageNet", "HMS", 2)

In [ ]:
#####################Feature Extraction and Model Creation####################

#Load Data (Features) for CE-MRI dataset
TrainX, TestX, TrainY, TestY = load_ce_mri_features("DA_TF_F1B")

#Create model_1
model_1 = create_model_1_F1B('CE-MRI')
model_1.compile(optimizer = optimizers.Adam(learning_rate=0.00005),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

#create callback
callbacks = []
checkpoint_path = base_path + 'Results/' + strategy + '/CE-MRI/Model 1/'
callbacks.append(ModelCheckpoint(checkpoint_path + 'checkpoint-{epoch}.h5'))


In [ ]:
#####################Training#################################
history = model_1.fit(TrainX, TrainY, batch_size=64, epochs=500, callbacks=callbacks)

In [ ]:
######################Evaluate on Test Set####################
test_set_auc = np.zeros((500,))
for i in range(1, 501):
  loaded_model = models.load_model(checkpoint_path + 'checkpoint-' + str(i) + '.h5')
  y_pred = loaded_model.predict(TestX, steps=TestX.shape[0], verbose=0)
  test_set_auc[i] = roc_auc_score(TestY, y_pred, average="macro")

#save test set AUC for all the epochs (1 to 500)
np.save(checkpoint_path + 'TestAUC.npy', test_set_auc)

In [ ]:
def prepare_train_test_sets(dataset, strategy):


In [ ]:
def fun():
    str = "geeksforgeeks"
    x   = 20
    temp = np.zeros((2, 2))
    new_temp = True
    return str, x, temp, new_temp;

In [ ]:
a, b, c, d = fun()
print (a)
print (b)
print (c)
print (d)

geeksforgeeks
20
[[0. 0.]
 [0. 0.]]
True
